### Feature engineering using orders and vendors information ###

During our initial analysis we decided to remove many columns from train_full. Here we will be adding in columns based on vendor and order information that matches with the customer and vendor in question. 

Below is a list of features we want to add: 
- Has ordered here before: binary (orders.csv) 
- Grand total: Average across customer’s order from vendor. (orders.csv) 
- Is_favorite: Set missing equal to 0, 1 otherwise. (orders.csv) 
- Distance between customer location and vendor location (train_full) 
- Number of times customer has ordered from this vendor tag (orders.csv) 
- One-hot encoding of vendor tag (train_full/vendors.csv) 
- Is_rated and vendor_rating (orders.csv) 

In [5]:
import pandas as pd
import numpy as np

# Read in CSV
orders = pd.read_csv('data/orders.csv')
vendors = pd.read_csv('data/vendors.csv')
training_data = pd.read_csv('data/sm_train_full.csv')
# training_data = pd.read_csv('data/lg_train_full.csv')

/var/folders/9t/xpbsmtsx0xzbs7_6z34b_5yr0000gn/T/ipykernel_11787/986147297.py:5: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('data/orders.csv')


Removing columns dropped from training datasets.

In [6]:
# Gathering the list of columns we removed from train_full.csv
columns = ['is_open', 'status_y', 'device_type', 'verified_y', 'commission', 'is_akeed_delivering', 'language', 'open_close_flags', 'one_click_vendor', 'country_id', 'city_id', 'display_orders', 'gender', 'location_type', 'OpeningTime', 'OpeningTime2', 'sunday_from_time1', 'sunday_to_time1', 'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1', 'monday_to_time1', 'monday_from_time2', 'monday_to_time2', 'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2', 'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1', 'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1', 'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2', 'friday_from_time1', 'friday_to_time1', 'friday_from_time2', 'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1', 'saturday_from_time2', 'saturday_to_time2', 'primary_tags', 'status_x', 'verified_x', 'created_at_x', 'updated_at_x', 'authentication_id', 'vendor_category_en', 'rank', 'created_at_y', 'updated_at_y']

# Drop columns from the dataframes if they were removed from train_full.csv
for col in columns:
    if col in orders.columns:
        orders = orders.drop(col, axis=1)
    if col in vendors.columns:
        vendors = vendors.drop(col, axis=1)
    
        
# Drop rows in orders where the location_number != 0
orders = orders[orders['LOCATION_NUMBER'] == 0]

print("Cleaned up dataframes columns")
print(orders.columns)
print(vendors.columns)

temp = training_data.copy()

print(training_data.columns)
# print(lg_train.columns)

# print(orders['LOCATION_NUMBER'].value_counts())
# print(orders['is_rated'].value_counts())
# print(orders['vendor_rating'].value_counts())

Cleaned up dataframes columns
Index(['akeed_order_id', 'customer_id', 'item_count', 'grand_total',
       'payment_mode', 'promo_code', 'vendor_discount_amount',
       'promo_code_discount_percentage', 'is_favorite', 'is_rated',
       'vendor_rating', 'driver_rating', 'deliverydistance', 'preparationtime',
       'delivery_time', 'order_accepted_time', 'driver_accepted_time',
       'ready_for_pickup_time', 'picked_up_time', 'delivered_time',
       'delivery_date', 'vendor_id', 'created_at', 'LOCATION_NUMBER',
       'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR'],
      dtype='object')
Index(['id', 'latitude', 'longitude', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'prepration_time', 'discount_percentage', 'status',
       'verified', 'vendor_rating', 'vendor_tag', 'vendor_tag_name',
       'created_at', 'updated_at'],
      dtype='object')
Index(['customer_id', 'location_number', 'latitude_x', 'longitude_x', 'id',
       'latitude_y', 'longitude_y', 'vendor_cat

In [7]:
# Constants to be used in the code
column_name = 'CID X LOC_NUM X VENDOR'

# Ensure the columns are treated as strings
orders[column_name] = orders[column_name].astype(str)
temp[column_name] = temp[column_name].astype(str)

Feature: Has Ordered Before
- Binary indicated customer has ordered from here before (1) or not (0).

In [8]:
# Add the 'ordered_before' column to dataframe and initialize with 0
temp['ordered_before'] = 0

# # Iterate through each row in orders dataframe
# for index, row in orders.iterrows():
#     if row[column_name] in orders[column_name].values:
#         temp.at[index, 'ordered_before'] = 1

# Replacing for-loop to make better use of pandas
temp = temp.merge(
    orders[[column_name]].drop_duplicates(),  # Assuming 'column_name' is the key linking temp and orders
    on=column_name,
    how='left',
    indicator=True
)
temp['ordered_before'] = (temp['_merge'] == 'both').astype(int)
temp.drop(columns=['_merge'], inplace=True) 


# Print the first few rows of the updated orders dataframe to verify
print(temp[['customer_id', 'id', 'ordered_before']].head(5))

# Print value counts for ordered before
print(temp['ordered_before'].value_counts())

  customer_id  id  ordered_before
0     TCHWPBT   4               0
1     TCHWPBT  13               0
2     TCHWPBT  28               0
3     TCHWPBT  33               0
4     TCHWPBT  43               0
ordered_before
0    423007
1      4398
Name: count, dtype: int64


Feature: Average total by customer and vendor
- Numeric value

In [9]:
# Create new columns
temp['vendor_average'] = 0
temp['customer_average'] = 0

# Calculate the average total cost by vendor and customer
vendor_avg = orders.groupby('vendor_id')['grand_total'].mean().to_dict()
customer_avg = orders.groupby('customer_id')['grand_total'].mean().to_dict()

print(len(customer_avg))

# Add the 'vendor_average' and 'customer_average' columns to dataframe
temp['vendor_average'] = temp['id'].map(vendor_avg).fillna(0)
temp['customer_average'] = temp['customer_id'].map(customer_avg).fillna(0)

print(temp[['customer_id', 'id', 'vendor_average', 'customer_average']].head(5))

25450
  customer_id  id  vendor_average  customer_average
0     TCHWPBT   4       16.856517               5.4
1     TCHWPBT  13       18.324214               5.4
2     TCHWPBT  28       12.308660               5.4
3     TCHWPBT  33       22.147441               5.4
4     TCHWPBT  43       12.038301               5.4


Feature: Favorite Vendor
- Binary. 1 if favorited, 0 if not or null.

In [10]:
# Add the column to dataframe and initialize with 0
temp['is_favorite'] = 0

# Filter to only include orders that have been rated
fav_orders = orders[orders['is_favorite'] == 'Yes']  # Filter orders marked as favorite

# Create temp dataset to merge with fav_orders
temp = temp.merge(
    fav_orders[[column_name]].drop_duplicates(),
    on=column_name,
    how='left',
    indicator=True
)
temp['is_favorite'] = (temp['_merge'] == 'both').astype(int)
temp.drop(columns=['_merge'], inplace=True)
        
print(temp[['customer_id', 'id', 'is_favorite']].head(5))
print(temp['is_favorite'].value_counts())

  customer_id  id  is_favorite
0     TCHWPBT   4            0
1     TCHWPBT  13            0
2     TCHWPBT  28            0
3     TCHWPBT  33            0
4     TCHWPBT  43            0
is_favorite
0    427331
1        74
Name: count, dtype: int64


Feature: Number of times customer has ordered from vendor

In [11]:
# Add the column to dataframe and initialize with 0
temp['times_ordered'] = 0

# Count number of times customer has ordered from a specific vendor
number_times = orders.groupby(column_name).size().to_dict()

# Map values to dataset
temp['times_ordered'] = temp[column_name].map(number_times).fillna(0).astype(int)

print(temp[['customer_id', 'id', 'times_ordered']].head(5))

target_counts = temp['times_ordered'].value_counts()
print("Counts for number of times ordered:")
print(target_counts)

  customer_id  id  times_ordered
0     TCHWPBT   4              0
1     TCHWPBT  13              0
2     TCHWPBT  28              0
3     TCHWPBT  33              0
4     TCHWPBT  43              0
Counts for number of times ordered:
times_ordered
0     423007
1       3175
2        660
3        247
4        114
5         82
6         49
7         20
8         19
11         8
9          7
10         6
14         2
17         2
13         2
12         2
51         1
22         1
15         1
Name: count, dtype: int64


Feature: Vendor Rating
- Binary if a customer has (1) or has not (0) rated the vendor.
- Vendor rating from customer, 0 if missing

In [12]:
# Add the columns to dataframe and initialize with 0
temp['is_rated'] = 0
temp['customer_rating'] = 0

# Filter orders where customer rated the vendor
rated_orders = orders[orders['is_rated'] == 'Yes']

# Rename column to avoid confusion
rated_orders.rename(columns={'vendor_rating': 'rating'}, inplace=True) 

# Merge rated_orders with temp dataset
temp = temp.merge(
    rated_orders[[column_name, 'rating']],  # Only include relevant columns
    on=column_name,
    how='left'
)

# Replace missing customer ratings with 0 (indicating no rating given)
temp['customer_rating'] = temp['rating'].fillna(0)

# Update 'is_rated' column to reflect whether the customer rated the vendor
temp['is_rated'] = temp['customer_rating'].apply(lambda x: 1 if x > 0 else 0)

temp.drop(columns=['rating'], inplace=True)

# Print results
print(temp[['customer_id', 'id', 'customer_rating', 'is_rated']].head(5))
print(temp['is_rated'].value_counts())
print(temp['customer_rating'].value_counts())
print(temp.columns)

/var/folders/9t/xpbsmtsx0xzbs7_6z34b_5yr0000gn/T/ipykernel_11787/675471179.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rated_orders.rename(columns={'vendor_rating': 'rating'}, inplace=True)


  customer_id  id  customer_rating  is_rated
0     TCHWPBT   4              0.0         0
1     TCHWPBT  13              0.0         0
2     TCHWPBT  28              0.0         0
3     TCHWPBT  33              0.0         0
4     TCHWPBT  43              0.0         0
is_rated
0    426875
1       672
Name: count, dtype: int64
customer_rating
0.0    426875
5.0       439
4.0       113
3.0        55
1.0        41
2.0        24
Name: count, dtype: int64
Index(['customer_id', 'location_number', 'latitude_x', 'longitude_x', 'id',
       'latitude_y', 'longitude_y', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'prepration_time', 'discount_percentage',
       'vendor_rating', 'vendor_tag', 'vendor_tag_name', 'location_number_obj',
       'id_obj', 'CID X LOC_NUM X VENDOR', 'target', 'ordered_before',
       'vendor_average', 'customer_average', 'is_favorite', 'times_ordered',
       'is_rated', 'customer_rating'],
      dtype='object')


Feature: Distance
- Distance from customer's location to vendor's location
- Serving distance from vendors.csv

_Note: Since the locations have been masked this may need to be normalized to a range to show some relationship._

In [13]:
# Initialize the distance column
temp['distance'] = 0

# Euclidean distance
temp['distance'] = np.sqrt((temp['latitude_x'] - temp['latitude_y'])**2 + (temp['longitude_x'] - temp['longitude_y'])**2)

print("Distance before transofrmation:")
print(temp['distance'].describe())

# Apply a log transformation
temp['distance'] = np.log1p(temp['distance'])

# Calculate the statistics for transformation
distance_min = temp['distance'].min()
distance_max = temp['distance'].max()
distance_mean = temp['distance'].mean()
distance_std = temp['distance'].std()

serving_distance_mean = vendors['serving_distance'].mean()
serving_distance_std = vendors['serving_distance'].std()

# Rescale the distance column to match serving_distance range [2, 15]
temp['distance_scaled'] = 2 + (temp['distance'] - distance_min) / (distance_max - distance_min) * (15 - 2)

# Match mean and std of serving_distance
temp['serving_distance'] = serving_distance_mean + ((temp['distance_scaled'] - temp['distance_scaled'].mean()) / 
                                                       temp['distance_scaled'].std()) * serving_distance_std

# Clip values to the maximum serving distance (optional)
temp['serving_distance'] = temp['serving_distance'].clip(upper=15)

print("Distance column information:")
print(temp['distance_scaled'].describe())

# Final summary statistics of the generated serving_distance column
print("Generated serving_distance column information:")
print(temp['serving_distance'].describe())

# Display the updated DataFrame
print(temp[['customer_id', 'id', 'distance', 'distance_scaled', 'serving_distance']].head(5))


Distance before transofrmation:
count    427547.000000
mean         33.978695
std          41.820776
min           0.000540
25%           0.703821
50%           1.491614
75%          79.047070
max        1044.847837
Name: distance, dtype: float64
Distance column information:
count    427547.000000
mean          6.057566
std           3.519171
min           2.000000
25%           2.995439
50%           3.706129
75%          10.194278
max          15.000000
Name: distance_scaled, dtype: float64
Generated serving_distance column information:
count    427547.000000
mean         11.087298
std           3.321123
min           7.013438
25%           8.175455
50%           9.005072
75%          15.000000
max          15.000000
Name: serving_distance, dtype: float64
  customer_id  id  distance  distance_scaled  serving_distance
0     TCHWPBT   4  4.774880        10.927795              15.0
1     TCHWPBT  13  4.775635        10.929208              15.0
2     TCHWPBT  28  4.781257        10.93972

Feature: Vendor Tags
If a customer has ordered a matching vendor tag to the vendor given. 
The number of matching tags in customers history to current vendor.

In [14]:
# New features
temp['ordered_vendor_tag'] = 0
temp['matching_tags_count'] = 0

vendors['vendor_tag'] = vendors['vendor_tag'].fillna('')  # Replace NaN with empty string

# Create mapping of vendor id to tags
# vendors['vendor_tag'] = vendors['vendor_tag'].apply(lambda x: set(map(int, x.split(','))) if x else set())
vendor_tags = dict(zip(vendors['id'], vendors['vendor_tag']))

# Tags customers have ordered
customer_tags = {}

# For each order, map the vendor's tags to the customer
for idx, row in orders.iterrows():
    customer_id = row['customer_id']
    vendor_id = row['vendor_id']
    if vendor_id in vendor_tags:  # Ensure the vendor exists in vendors_df
        tags = vendor_tags[vendor_id]
        if customer_id not in customer_tags:
            customer_tags[customer_id] = set(tags)
        else:
            customer_tags[customer_id].update(tags)

def calculate_tag_columns(row):
    customer_id = row['customer_id']
    vendor_id = row['id']
    
    # Get the vendor's tags
    vendor_tags_set = vendor_tags.get(vendor_id, set())
    
    # Get the customer's ordered tags
    customer_tags_set = customer_tags.get(customer_id, set())
    
    # Calculate the matching tags
    matching_tags = customer_tags_set.intersection(vendor_tags_set)
    ordered_vendor_tag = 1 if matching_tags else 0
    matching_tags_count = len(matching_tags)
    
    return pd.Series({
        'ordered_vendor_tag': ordered_vendor_tag,
        'matching_tags_count': matching_tags_count
    })

# Apply the function to train_full_df
temp[['ordered_vendor_tag', 'matching_tags_count']] = temp.apply(calculate_tag_columns, axis=1)

print(temp.head())
print(temp[['customer_id', 'id', 'ordered_vendor_tag', 'matching_tags_count']].head(5))

  customer_id  location_number  latitude_x  longitude_x  id  latitude_y  \
0     TCHWPBT                0      -96.44        -67.2   4     -0.5884   
1     TCHWPBT                0      -96.44        -67.2  13     -0.4717   
2     TCHWPBT                0      -96.44        -67.2  28      0.4807   
3     TCHWPBT                0      -96.44        -67.2  33     -0.4946   
4     TCHWPBT                0      -96.44        -67.2  43     -0.1150   

   longitude_y  vendor_category_id  delivery_charge  serving_distance  ...  \
0       0.7544                 2.0              0.0              15.0  ...   
1       0.7446                 2.0              0.7              15.0  ...   
2       0.5527                 2.0              0.7              15.0  ...   
3       0.7430                 2.0              0.7              15.0  ...   
4       0.5460                 2.0              0.7              15.0  ...   

   vendor_average  customer_average  is_favorite times_ordered is_rated  \
0    

The locations being transformed this way may become an issue, but it now more closely represents the findings in 'serving_distance'

Before saving the dataset we will remove: 'vendor_tag', 'vendor_tag_name', 'CID X LOC_NUM X VENDOR' and make sure out small and large dataset contain the same columns. 

In [15]:
# Drop columns that are no longer needed
final_drop_cols = ['location_number_obj', 'id_obj','vendor_tag', 'vendor_tag_name', 'CID X LOC_NUM X VENDOR', 'location_number']

# Rename id to vendor_id
temp = temp.rename(columns={'id': 'vendor_id'})

for col in final_drop_cols:
    if col in temp.columns:
        temp = temp.drop(col, axis=1)
    
print("Final columns in dataframe:")
print(temp.columns)
print(temp.shape)

# Save the updated dataframe to a new CSV file
# temp.to_csv('data/new_sm_train.csv', index=False) # For smaller dataset
# temp.to_csv('data/new_train.csv', index=False) # For larger dataset

# Confirm that the number of columns in the new CSV file is the same as the original
new_sm = pd.read_csv('data/new_sm_train.csv')
print(len(new_sm.columns) == len(temp.columns))

Final columns in dataframe:
Index(['customer_id', 'latitude_x', 'longitude_x', 'vendor_id', 'latitude_y',
       'longitude_y', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'prepration_time', 'discount_percentage',
       'vendor_rating', 'target', 'ordered_before', 'vendor_average',
       'customer_average', 'is_favorite', 'times_ordered', 'is_rated',
       'customer_rating', 'distance', 'distance_scaled', 'ordered_vendor_tag',
       'matching_tags_count'],
      dtype='object')
(427547, 24)
True
